In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import os

os.environ["HADOOP_USER_NAME"] = "hdfs"
os.environ["PYTHON_VERSION"] = "3.5.2"
SUBMIT_ARGS = "--packages org.elasticsearch:elasticsearch-spark-20_2.11:6.2.1 pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = SUBMIT_ARGS

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("File lines count") \
    .enableHiveSupport() \
    .config("hive.metastore.uris", "thrift://nn1:9083") \
    .config("spark.driver.cores", "1") \
    .config("spark.driver.memory", "1g") \
    .config("spark.cores.max", "20") \
    .config("spark.executor.cores", "1") \
    .config("spark.executor.memory", "2G") \
    .config("spark.executor.heartbeatInterval", "90s") \
    .config("spark.network.timeout", "240s") \
    .config("spark.blacklist.enabled", "false") \
    .config("spark.python.worker.reuse", "true") \
    .config("spark.mesos.executor.docker.forcePullImage","false") \
    .config("es.nodes.wan.only", "true") \
    .getOrCreate() 

In [ ]:
df = spark.sql("""SELECT * FROM onboarding_formateur.cart 
INNER JOIN onboarding_formateur.cartproducts on (cart.id = cartproducts.cart_id)
INNER JOIN onboarding_formateur.product on (cartproducts.id = product.id)""")

In [ ]:
df.show()

In [ ]:
df.write \
    .format("org.elasticsearch.spark.sql") \
    .option("es.resource", '%s/%s' % ("my_cart", "my_doc")) \
    .option("es.nodes", "datamart2") \
    .option("es.port", "9200") \
    .save()